In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os,glob
import pandas as pd
import numpy as np
import ast
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from keras.layers import Flatten
from keras.models import Model
#import warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
# df=pd.read_csv('articles.csv')

df=pd.read_csv('drive/MyDrive/Deep_Learning/guardian_articles.csv')
# dropping the rows having NaN values
df = df.dropna()

In [3]:
df

,article_id,sectionName,webTitle,webUrl,bodyContent,webPublicationDate,id
0,us-news/2016/jan/31/iowa-caucus-underdog-candi...,US news,Iowa underdogs put on brave faces despite all ...,https://www.theguardian.com/us-news/2016/jan/3...,As polling day looms and the cameras turn only...,2016-01-31T23:53:37Z,1
1,us-news/2016/jan/31/iowa-caucus-worlds-most-pa...,US news,Iowa caucus: hologram eagle and Jesus star on ...,https://www.theguardian.com/us-news/2016/jan/3...,"In Des Moines on Sunday, the Guardian was give...",2016-01-31T23:46:28Z,2
2,world/2016/jan/31/tanzania-britsh-helicopter-p...,World news,British pilot in Tanzania 'manoeuvred ​to save...,https://www.theguardian.com/world/2016/jan/31/...,A British pilot who was shot dead by an elepha...,2016-01-31T23:43:48Z,3
3,football/2016/jan/31/late-winner-gets-usa-off-...,Football,USA 3-2 Iceland | International friendly match...,https://www.theguardian.com/football/2016/jan/...,USA took a step toward shaking off the ghosts ...,2016-01-31T23:30:49Z,4
4,football/2016/jan/31/blackburn-paul-lambert-ox...,Football,Reinvigorated Paul Lambert reflects after impr...,https://www.theguardian.com/football/2016/jan/...,"The clean-shaven, spectacle free and suspiciou...",2016-01-31T22:30:10Z,5
...,...,...,...,...,...,...,...
149834,world/2022/jun/21/marble-head-of-hercules-pull...,World news,Marble head of Hercules pulled up from Roman s...,https://www.theguardian.com/world/2022/jun/21/...,"For archaeologists, it’s the underwater find t...",2022-06-21T17:31:32Z,149835
149835,music/2022/jun/22/i-got-sick-of-talking-about-...,Music,‘I got sick of talking about myself’: Spacey J...,https://www.theguardian.com/music/2022/jun/22/...,"From under a mop of curls, Caleb Harper – Spac...",2022-06-21T17:30:09Z,149836
149836,australia-news/2022/jun/22/the-small-town-with...,Australia news,The small town with a big potato that inspired...,https://www.theguardian.com/australia-news/202...,"Robertson is a small, pretty town perched on t...",2022-06-21T17:30:09Z,149837
149837,australia-news/2022/jun/22/power-to-ban-citize...,Australia news,Power to ban citizens from re-entering Austral...,https://www.theguardian.com/australia-news/202...,A high court decision striking down the home a...,2022-06-21T17:30:08Z,149838


In [4]:
df['text']=df['bodyContent']
df['sentiment']=df['sectionName']

In [5]:
vc = df['sentiment'].value_counts()  # get value counts of 'class' column
keep_classes = vc.index[vc >= 10000]  # get classes with counts >= 2
df = df[df['sentiment'].isin(keep_classes)]

In [6]:
df=df[:5000]

In [7]:
train_size = int(len(df) * 0.8)
train_data = df['text'][:train_size]
train_labels = df['sentiment'][:train_size]
test_data = df['text'][train_size:]
test_labels = df['sentiment'][train_size:]

In [8]:
# Fit and transform the training data with the TF-IDF vectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_data)

In [9]:
# Transform the test data using the fitted vectorizer
X_test = vectorizer.transform(test_data)

In [11]:
# Fit the SVM model on the training data
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, train_labels)

RandomForestClassifier(random_state=42)

In [13]:
# Evaluate the model on the test data
acc =rf_classifier.score(X_test, test_labels)
print("Accuracy:", acc)

Accuracy: 0.912


In [14]:
from keras.models import Sequential, load_model
import pickle
# Save the trained model
filename = 'Random_forest_model.sav'
pickle.dump(rf_classifier, open(filename, 'wb'))
# Load the saved model
loaded_model = pickle.load(open(filename, 'rb'))
y_pred = loaded_model.predict(X_test)
print(y_pred)

['World news' 'Opinion' 'Opinion' 'Sport' 'Sport' 'Opinion' 'World news'
 'Opinion' 'World news' 'Opinion' 'World news' 'World news' 'World news'
 'Sport' 'Opinion' 'Football' 'Opinion' 'Football' 'Football' 'Sport'
 'Opinion' 'Opinion' 'World news' 'World news' 'Sport' 'Football'
 'Football' 'Opinion' 'Football' 'Football' 'World news' 'Football'
 'Sport' 'World news' 'Opinion' 'World news' 'Football' 'Football'
 'Football' 'Opinion' 'World news' 'World news' 'World news' 'Opinion'
 'Opinion' 'Sport' 'World news' 'Opinion' 'World news' 'Opinion' 'Opinion'
 'World news' 'Sport' 'Sport' 'Football' 'Football' 'Sport' 'Sport'
 'Sport' 'Sport' 'World news' 'Sport' 'Football' 'Sport' 'World news'
 'Football' 'Football' 'Sport' 'World news' 'Opinion' 'World news'
 'World news' 'World news' 'Opinion' 'World news' 'Sport' 'Sport' 'Sport'
 'Opinion' 'World news' 'Football' 'World news' 'World news' 'Football'
 'Football' 'Football' 'Opinion' 'Opinion' 'Football' 'Opinion' 'Football'
 'World new